In [ ]:
import numpy as np
from tensorflow import keras
import os
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dropout, Activation, UpSampling2D, GlobalMaxPooling2D, multiply
from tensorflow.keras.backend import max
from tensorflow.keras.optimizers import Adam
import Task1_handling
import openslide
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.activations import relu

In [ ]:
from keras_unet_collection import models, losses
#!pip install tensorflow_addons
import tensorflow_addons as tfa
import keras.backend as K
import tensorflow as tf

In [ ]:
data=np.load('../dataset/dataset_new_train.npz')

In [ ]:
org_img=data['x']
layer3=data['y']
layer5=data['z']

In [ ]:
y=np.stack((layer3,layer5),axis=-1)/255
y.shape

In [ ]:
X=org_img
X_norm=np.zeros(X.shape)

In [ ]:
#normalizaiton
for i in range(len(X_norm)):
    X_norm[i]=keras.utils.normalize(X[i]/255,axis=-1)

  #train,test split
from sklearn.model_selection import train_test_split
np.random.seed(seed=2021)

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, shuffle=False)

In [ ]:
train_layer3=y_train[:,:,:,0] #.reshape(-1,256,256,1)
train_layer5=y_train[:,:,:,1] #.reshape(-1,256,256,1)

test_layer3=y_test[:,:,:,0] #.reshape(-1,256,256,1)
test_layer5=y_test[:,:,:,1] #.reshape(-1,256,256,1)

In [ ]:
model_pretrained=load_model('../pretrained/att_unet_pretrained.hdf5', compile=False)

model_pretrained.summary()

In [ ]:
from keras.models import Model
model_main_branch=Model(inputs=model_pretrained.input,outputs=model_pretrained.layers[-3].output)

In [ ]:
from tensorflow.keras.layers import Conv2D,Activation
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
#layer3_branch=Input(model_pretrained.layers[-3].output.shape)
layer3_Conv=Conv2D(filters=1, kernel_size=3, padding='Same', activation='relu')(model_main_branch.output)
layer3_sig=Activation('sigmoid',name='layer3')(layer3_Conv)

layer5_Conv=Conv2D(filters=1, kernel_size=3, padding='Same', activation='relu')(model_main_branch.output)
layer5_sig=Activation('sigmoid',name='layer5')(layer5_Conv)

In [ ]:
multi_task=Model(inputs=model_main_branch.input, outputs=[layer3_sig,layer5_sig])

In [ ]:
multi_task.compile(loss={'layer3':layer3_loss,'layer5':layer5_loss},
                   loss_weights={'layer3':0.5, 'layer5':0.5}
                   ,optimizer=Adam(learning_rate = 1e-4), 
                  metrics={'layer3':['accuracy',losses.dice_coef],
                          'layer5':['accuracy',losses.dice_coef]})

In [ ]:
Unet_history = multi_task.fit(X_train, [train_layer3,train_layer5], 
                    verbose=1,
                    batch_size = 16,
                    validation_data=(X_test, [test_layer3,test_layer5]), 
                    shuffle=False,
                    epochs=50)